In [ ]:
import smbus2
import time
from IPython.display import clear_output

In [ ]:
addr_switch = 0x70
addr_switch = 0x7c
addr_charger = 0x6a
addr_rtc = 0x51

In [ ]:
c.close()

In [ ]:
def read_2(c, addr, reg):
    d = c.read_i2c_block_data(addr, reg, 2)
    #print([bin(x) for x in d])

    d = d[0]| d[1] << 8
    #print(bin(d))
    return d

def crop(number, start=0, end=16):
    # Vytvoření masky s jedničkami na pozicích od start do end
    mask = ((1 << (end - start + 1)) - 1) << start
    
    # Použití masky pro vybrání příslušných bitů
    result = (number & mask) >> start
    
    return result

def comp(data, length=16):

    if data & (1 << (length-1)):
        data = data ^ ((1 << length) - 1)
        data = -(data + 1)

    return data



In [ ]:
bin(crop(0b00101100, 2, 6))

In [ ]:
def con():
    return smbus2.SMBus(19)

c = con()

# I2C switch, switch internal I2C bus into USB (FT260) IO

In [ ]:
c=con()

In [ ]:
c.write_byte_data(addr_switch, 0x01, 0x03) # nastavit I2C switch do USB

In [ ]:
hex(c.read_byte(addr_switch, 0x01))

# Nabíječka 

In [ ]:
# Vypnout bateriovy modul
c.write_byte_data(addr_charger, 0x18, 0x0a)

In [ ]:
print(bin(c.read_byte_data(addr_charger, 0x16)))


charge_current_ma = 800

c.write_byte_data(addr_charger, 0x02, int(charge_current_ma/40)<<5)
c.write_byte_data(addr_charger, 0x14, 0b00100010)
c.write_byte_data(addr_charger, 0x15, 0b00011101)
c.write_byte_data(addr_charger, 0x16, 0b10100000)
c.write_byte_data(addr_charger, 0x17, 0b01010110)
c.write_byte_data(addr_charger, 0x18, 0b00000000)
c.write_byte_data(addr_charger, 0x19, 0b00000001)


# NTC
c.write_byte_data(addr_charger, 0x1a, 0b10111111)

# ADC
c.write_byte_data(addr_charger, 0x26, 0b10001100)

In [ ]:

while True:
    # Zapnout interni ADC prevodnik
    #print(bin(c.read_byte_data(addr_charger, 0x26)))
    #c.write_byte_data(addr_charger, 0x26, 0b10001100)
    
    clear_output()
    print("IBUS ADC", comp(crop(read_2(c, addr_charger, 0x28), 1, 16), 15) * 2, " mA")
    #print(" ")

    print("IBAT ADC", comp(crop(read_2(c, addr_charger, 0x2A), 2, 16), 14) * 4, " mA")
    #print(" ")

    print("VBUS ADC", crop(read_2(c, addr_charger, 0x2C), 2, 15) * 3.97, " mV")
    #print(" ")

    print("VPMID ADC", crop(read_2(c, addr_charger, 0x2E), 2, 15) * 3.97, " mV")
    #print(" ")

    print("VBAT ADC", crop(read_2(c, addr_charger, 0x30), 1, 13) * 1.99, " mV")
    #print(" ")

    print("VSYS ADC", crop(read_2(c, addr_charger, 0x32), 1, 13) * 1.99, " mV")
    #print(" ")

    print("TS ADC", crop(read_2(c, addr_charger, 0x34), 0, 12) * 0.0961, " %")
    #print(" ")

    print("TDIE ADC", crop(read_2(c, addr_charger, 0x36), 0, 12) * 0.5 - 40, " C")
    #print(" ")

    time.sleep(1)


In [ ]:
# Nastavit timery
c.write_byte_data(addr_charger, 0x15, 0b00001001)

In [ ]:
d = c.read_byte_data(addr_charger, 0x1d)
print("Charger status 0: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x1e)
print("Charger status 1: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x1f)
print("Fault reg 0: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x20)
print("Charger flag 0: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x21)
print("Charger flag 1: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x22)
print("Fault flag 0: 0b{0:08b}".format(d))


# RTC

In [ ]:
import datetime

In [ ]:
def get_time():
    r01 = c.read_byte_data(addr_rtc, 0x01)
    r02 = c.read_byte_data(addr_rtc, 0x02)
    r03 = c.read_byte_data(addr_rtc, 0x03)
    r04 = c.read_byte_data(addr_rtc, 0x04)
    r06 = c.read_byte_data(addr_rtc, 0x06)
    r07 = c.read_byte_data(addr_rtc, 0x07)

    sec = ((r01 >> 4) & 0b111) * 10 + (r01 & 0b1111)
    minu= ((r02 >> 4) & 0b111) * 10 + (r02 & 0b1111)
    hour= ((r03 >> 4) & 0b11) * 10 + (r03 & 0b1111)
    day = ((r04 >> 4) & 0b11) * 10 + (r04 & 0b1111)
    mon = ((r06 >> 4) & 0b1) * 10 + (r06 & 0b1111)
    year= ((r07 >> 4) & 0b1111) * 10 + (r07 & 0b1111)+2000
        
    return datetime.datetime(year, mon, day, hour, minu, sec)
    
def set_time(time):
    c.write_byte_data(addr_rtc, 0x01, (time.second % 10) | int(time.second/10)<<4 )
    c.write_byte_data(addr_rtc, 0x02, (time.minute % 10) | int(time.minute/10)<<4 )
    c.write_byte_data(addr_rtc, 0x03, (time.hour % 10) | int(time.hour/10)<<4 )
    c.write_byte_data(addr_rtc, 0x04, (time.day % 10) | int(time.day/10)<<4 )
    c.write_byte_data(addr_rtc, 0x06, (time.month % 10) | int(time.month/10)<<4 )
    c.write_byte_data(addr_rtc, 0x07, ((time.year-2000) % 10) | int((time.year-2000)/10)<<4 )

In [ ]:
actual_time = datetime.datetime.utcnow()
detector_time = get_time()
print("Actual time", actual_time)
print("Detector time", detector_time)
print("Difference", detector_time-actual_time)

In [ ]:
actual_time = datetime.datetime.utcnow()
set_time(actual_time)

In [ ]:
actual_time.second

In [ ]:

time = actual_time

In [ ]:
bin( (time.second % 10)<<4)